# sys.argv

In [3]:
import sys

arguments = sys.argv
print(arguments)

script_path = sys.argv[0]
print(script_path)
print(sys.argv[1])

['c:\\users\\tiezh\\appdata\\local\\programs\\python\\python38-32\\lib\\site-packages\\ipykernel_launcher.py', '-f', 'C:\\Users\\tiezh\\AppData\\Roaming\\jupyter\\runtime\\kernel-d34a97d1-1440-493b-8533-b5781d228e08.json']
c:\users\tiezh\appdata\local\programs\python\python38-32\lib\site-packages\ipykernel_launcher.py
-f


# configuration file

example: separator could be colon : or equal =

[DEFAULT]
host = localhost # This is a comment.

[mariadb]
name = hello
user = user
password = password

[redis]
port = 6379
db = 0

In [11]:
import configparser

#read configuration form text file
config = configparser.ConfigParser()
config.read('config.ini')

print('Sections:', config.sections(),'\n')

print('mariadb section:')
print('Host:', config['mariadb']['host'])
print('Database:', config['mariadb']['name'])
print('Username:', config['mariadb']['user'])
print('Password:', config['mariadb']['password'], '\n')

print('redis section:')
print('Host:', config['redis']['host'])
print('Port:', int(config['redis']['port']))
print('Database number:', int(config['redis']['db']))


mariadb section:
Host: localhost # This is a comment.
Database: hello
Username: user
Password: password 

redis section:
Host: localhost # This is a comment.
Port: 6379
Database number: 0


In [8]:
import configparser

dict = {
    'DEFAULT': {
        'host': 'localhost'
    },
    'mariadb': {
        'name': 'hello',
        'user': 'root',
        'password': 'password'
    },
    'redis': {
        'port': 6379,
        'db': 0
    }
}

#read configuration from python dictionary
config = configparser.ConfigParser()
config.read_dict(dict)

print('Sections:', config.sections(),'\n')

print('mariadb section:')
print('Host:', config['mariadb']['host'])
print('Database:', config['mariadb']['name'])
print('Username:', config['mariadb']['user'])
print('Password:', config['mariadb']['password'], '\n')

print('redis section:')
print('Host:', config['redis']['host'])
print('Port:', int(config['redis']['port']))
print('Database number:', int(config['redis']['db']))

Sections: ['mariadb', 'redis'] 

mariadb section:
Host: localhost
Database: hello
Username: root
Password: password 

redis section:
Host: localhost
Port: 6379
Database number: 0


In [ ]:
import configparser

#create a configuration text file
config = configparser.ConfigParser()

config['DEFAULT'] = {'host': 'localhost'}
config['mariadb'] = {'name': 'hello', 'user': 'root',
                     'password': 'password'}
config['redis'] = {'port': 6379, 'db': 0}

with open('config.ini', 'w') as configfile:
    config.write(configfile)

## interpolate configuration values

%(<key>)s
dsn = redis://%(host)s:%(port)s

For the dsn option, it'll be the following string: redis://localhost. Note that the placeholder %(host)s has been replaced by the value stored in the host option.

In [13]:
import configparser

config = configparser.ConfigParser()
config.read('config.ini')

print(config['redis']['dsn'])

redis://localhost:6379


## Scenario
Imagine a situation in which you receive a configuration file containing access data for various services. Unfortunately, the file is a terrible mess, because it contains data used in both production and development environments.

Your task will be to create two files named prod_config.ini and dev_config.ini. The prod_config.ini file should only contain sections for the production environment, while dev_config.ini should only contain sections for the development environment.

To distinguish between the environments, use the env option added to all sections in the mess.ini file. The env option should be removed from the sections before moving them to the files.

In [34]:
from configparser import ConfigParser as cp
from copy import deepcopy

def to_config_dict(config_file):
    config = cp()
    config.read(config_file)
    #print(config.sections())
    print(config.__dict__)
    config_dict = deepcopy(config._sections)
    config_dict['DEFAULT'] = config._defaults
    return config_dict

def to_config_file(config_dict, config_file):
    config = cp()
    config.read_dict(config_dict)
    with open(config_file, 'w') as f:
        config.write(f)

#
config_dict = to_config_dict('config.ini')
print(config_dict)
config_dict['dynamodb'] = { 'table': 'user', 
    'endpoint': f"{config_dict['redis']['port']}"} 
to_config_file(config_dict, 'config2.ini')
print("done")

{'_dict': <class 'dict'>, '_sections': {'mariadb': {'name': 'hello', 'user': 'user', 'password': 'password'}, 'redis': {'port': '6379', 'db': '0'}}, '_defaults': {'host': 'localhost # This is a comment.'}, '_converters': <configparser.ConverterMapping object at 0x058079E8>, '_proxies': {'DEFAULT': <Section: DEFAULT>, 'mariadb': <Section: mariadb>, 'redis': <Section: redis>}, '_delimiters': ('=', ':'), '_optcre': re.compile('\n        (?P<option>.*?)                    # very permissive!\n        \\s*(?P<vi>=|:)\\s*              # any number of space/tab,\n                                           # followed by any of t, re.VERBOSE), '_comment_prefixes': ('#', ';'), '_inline_comment_prefixes': (), '_strict': True, '_allow_no_value': False, '_empty_lines_in_values': True, 'default_section': 'DEFAULT', '_interpolation': <configparser.BasicInterpolation object at 0x003CB070>}
{'mariadb': {'name': 'hello', 'user': 'user', 'password': 'password'}, 'redis': {'port': '6379', 'db': '0'}, 'DEFA

In [46]:
from configparser import ConfigParser as cp
from copy import deepcopy
import os

def to_config_dict(config_file):
    config = cp()
    config.read(config_file)
    config_dict = deepcopy(config._sections)
    config_dict['DEFAULT'] = config._defaults
    return config_dict

def to_config_file(config_dict, config_file):
    config = cp()
    config.read_dict(config_dict)
    with open(config_file, 'w') as f:
        config.write(f)

#create configuration files
config_prod={'sentry':{'key':'key', 'secret':'secret'}, 
             'github':{'user':'user', 'password':'password'}}
to_config_file(config_prod, 'prod_config.ini')

config_dev={
    'DEFAULT': {'host':'localhost'},
    'mariadb':{'name':'hello', 'user': 'user', 'passward':'password'}, 
    'redis':{'port':'6379', 'db':'0'}}
to_config_file(config_dev, 'dev_config.ini')

#get configurations
os.environ['ENV']= 'dev'
config_dict = to_config_dict(f"{os.getenv('ENV')}_config.ini")
print(config_dict)
os.environ['ENV']= 'prod'
config_dict = to_config_dict(f"{os.getenv('ENV')}_config.ini")
print(config_dict)
print("done")

{'mariadb': {'name': 'hello', 'user': 'user', 'passward': 'password'}, 'redis': {'port': '6379', 'db': '0'}, 'DEFAULT': {'host': 'localhost'}}
{'sentry': {'key': 'key', 'secret': 'secret'}, 'github': {'user': 'user', 'password': 'password'}, 'DEFAULT': {}}
done
